<a href="https://colab.research.google.com/github/Vaycold/Python_ML/blob/main/Pattern_Recognition/%EA%B3%B5%EC%A0%95%EB%8D%B0%EC%9D%B4%ED%84%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 라이브러리

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from google.colab import drive

In [ ]:
drive.mount('/content/Drive')

Mounted at /content/Drive


In [ ]:
os.chdir('/content/')  
path = ('Drive/MyDrive/fastcampus/MachineLearning_300/Part 3 - 머신러닝과 패턴인식/Chapter 1. 철판 제조 공정 데이터를 활용한 분류모형 생성 및 성능 비교/data')
file_list = os.listdir(path)
print( file_list)

['Faults.NNA', 'Faults27x7_var']


In [ ]:
file_list[1]

'Faults27x7_var'

In [ ]:
# workind directory 설정
os.chdir('/content/' + path)

In [ ]:
df = pd.read_csv(file_list[0], delimiter = '\t', header=None)
df.head()

0     1        2        3     4   5    6   ...  27  28  29  30  31  32  33
0    42    50   270900   270944   267  17   44  ...   1   0   0   0   0   0   0
1   645   651  2538079  2538108   108  10   30  ...   1   0   0   0   0   0   0
2   829   835  1553913  1553931    71   8   19  ...   1   0   0   0   0   0   0
3   853   860   369370   369415   176  13   45  ...   1   0   0   0   0   0   0
4  1289  1306   498078   498335  2409  60  260  ...   1   0   0   0   0   0   0

[5 rows x 34 columns]

In [ ]:
# 요게 컬럼명인 파일이넹
attributes_name = pd.read_csv(file_list[1], delimiter = ' ',header=None)
attributes_name.head()

0
0     X_Minimum
1     X_Maximum
2     Y_Minimum
3     Y_Maximum
4  Pixels_Areas

In [ ]:
# 데이터 프레임에 0~33인 컬럼명을 저 데이터프레임으로 바꾸자
attributes_name[0].values

array(['X_Minimum', 'X_Maximum', 'Y_Minimum', 'Y_Maximum', 'Pixels_Areas',
       'X_Perimeter', 'Y_Perimeter', 'Sum_of_Luminosity',
       'Minimum_of_Luminosity', 'Maximum_of_Luminosity',
       'Length_of_Conveyer', 'TypeOfSteel_A300', 'TypeOfSteel_A400',
       'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index',
       'Square_Index', 'Outside_X_Index', 'Edges_X_Index',
       'Edges_Y_Index', 'Outside_Global_Index', 'LogOfAreas',
       'Log_X_Index', 'Log_Y_Index', 'Orientation_Index',
       'Luminosity_Index', 'SigmoidOfAreas', 'Pastry', 'Z_Scratch',
       'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults'],
      dtype=object)

In [ ]:
df.columns = attributes_name[0].values #values를 안넣어도 무관하긴하다
df.head() 

X_Minimum  X_Maximum  Y_Minimum  ...  Dirtiness  Bumps  Other_Faults
0         42         50     270900  ...          0      0             0
1        645        651    2538079  ...          0      0             0
2        829        835    1553913  ...          0      0             0
3        853        860     369370  ...          0      0             0
4       1289       1306     498078  ...          0      0             0

[5 rows x 34 columns]

In [ ]:
df.shape

(1941, 34)

In [ ]:
# cpu 갯수
n_cpu = os.cpu_count()
n_thread = n_cpu * 2

## EDA 및 PreProcessing

### 종속변수를 범주화 하자

In [ ]:
conditions = [df['Pastry'].astype(bool),
              df['Z_Scratch'].astype(bool),
              df['K_Scatch'].astype(bool),
              df['Stains'].astype(bool),
              df['Dirtiness'].astype(bool),
              df['Bumps'].astype(bool),
              df['Other_Faults'].astype(bool)]

In [ ]:
conditions

In [ ]:
# 이 방법으로 하는 것을 더 추천
condition_bf = [
                df['Pastry'],
                df['Z_Scratch'],
                df['K_Scatch'],
                df['Stains'],
                df['Dirtiness'],
                df['Bumps'],
                df['Other_Faults'],
] # 이렇게하면 pastry는 0번째 series, Z_scatch는 1번째 series, ... , otehr_Faults는 마지막 series가 된다.

conditions = list(map(lambda x : x.astype(bool), conditions))

In [ ]:
conditions[0] # True ,False로 이루어진 Series가 list로 구성됌

0        True
1        True
2        True
3        True
4        True
        ...  
1936    False
1937    False
1938    False
1939    False
1940    False
Name: Pastry, Length: 1941, dtype: bool

In [ ]:
print(type(conditions))
print(type(conditions[0]))
print(len(conditions))
print(len(conditions[0]))

<class 'list'>
<class 'pandas.core.series.Series'>
7
1941


In [ ]:
choice = ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains',
       'Dirtiness', 'Bumps', 'Other_Faults']

In [ ]:
df['class'] = np.select(conditions, choice) # 이건 돌면서 